In [1]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [2]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
name = 'parallel_test'
path = f'../persistent_data/{name}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,correct_alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,f1_cfa,scenario,ngram,algorithm,detection_time,window
0,55846,890,1230072,139035,172,174,0.988506,1,0,0.988506,0.994220,0.161959,0.991354,CVE-2017-7529/,3,stide,1.895963,NaN
1,52023,790,1230172,142858,172,174,0.988506,1,0,0.988506,0.994220,0.178794,0.991354,CVE-2017-7529/,3,stide,1.896127,NaN
2,58773,990,1229972,136108,172,174,0.988506,1,0,0.988506,0.994220,0.148021,0.991354,CVE-2017-7529/,3,stide,1.899889,NaN
3,60905,1090,1229872,133976,172,174,0.988506,1,0,0.988506,0.994220,0.136292,0.991354,CVE-2017-7529/,3,stide,1.913732,NaN
4,62580,1223,1229739,132301,172,174,0.988506,0,0,0.988506,1.000000,0.123297,0.994220,CVE-2017-7529/,3,stide,1.898429,NaN
5,51679,802,1228346,143202,172,174,0.988506,1,0,0.988506,0.994220,0.176591,0.991354,CVE-2017-7529/,5,stide,1.914086,NaN
6,55502,902,1228246,139379,172,174,0.988506,1,0,0.988506,0.994220,0.160149,0.991354,CVE-2017-7529/,5,stide,1.904165,NaN
7,62092,1190,1229772,132789,172,174,0.988506,1,0,0.988506,0.994220,0.126285,0.991354,CVE-2017-7529/,3,stide,1.907744,NaN
8,58429,1002,1228146,136452,172,174,0.988506,1,0,0.988506,0.994220,0.146508,0.991354,CVE-2017-7529/,5,stide,1.896537,NaN
9,62238,1235,1227913,132643,172,174,0.988506,0,0,0.988506,1.000000,0.122246,0.994220,CVE-2017-7529/,5,stide,1.899197,NaN


In [5]:
conn = sql.connect('stide_mode_flag_v2.db')
try:
    result_list.to_sql(name, conn)
except ValueError:
    print('Table already exists')

In [6]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])
name = name[0]

prallel_test
parallel_test


In [8]:
result = pd.read_sql(f'SELECT ngram, \
                              window, \
                              AVG(detection_rate) as DR, \
                              AVG(consecutive_false_positives_exploits)+ AVG(consecutive_false_positives_normal) as CFP_sum, \
                              Count(*), \
                              detection_time as time_in_min \
                              FROM  {name} \
                     GROUP BY ngram, window \
                     ORDER BY DR desc, \
                              CFP_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,ngram,window,DR,CFP_sum,Count(*),time_in_min
0,3,600.0,1.000000,6.000000,1,850.827383
1,3,700.0,1.000000,13.000000,1,874.183455
2,3,800.0,1.000000,14.000000,1,871.536257
3,3,1000.0,1.000000,20.000000,1,847.423292
4,5,600.0,1.000000,21.000000,1,922.932647
5,3,500.0,1.000000,22.000000,1,837.005401
6,3,900.0,1.000000,28.000000,1,844.829438
7,7,NaN,0.988506,0.833333,6,1.902200
8,5,NaN,0.988506,0.833333,6,1.914086
9,3,NaN,0.988506,0.833333,6,1.895963
